## Чистая версия DNN

На прошлой итерации очень хорошие результаты с DNN:
- val acc: 0.8161
- test acc: 0.8276

Kaggle LB: 0.80967

Optuna оптимизация толком не дала ничего, видимо, из-за переобучения под валидационный набор. Тут попробуем написать чистый код для формирования DNN и улучшить показатели вручную.

In [1]:
from IPython.lib.deepreload import reload
from keras.src.layers import BatchNormalization
from sklearn.linear_model import LogisticRegression
from torch.ao.nn.quantized import Dropout
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import optuna
import os
import gc
import pandas as pd
import random
import tensorflow as tf
import warnings

from optuna.integration import TFKerasPruningCallback
from optuna.samplers import TPESampler
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from tensorflow.keras.initializers import GlorotUniform, RandomNormal
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.regularizers import l1, l2, l1_l2



from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split, StratifiedKFold

from utils.data_manager import DataManager
from utils.helpers import save_features, save_data
from utils.model_manager import ModelManager

# Подавляем информационные логи TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

In [2]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [3]:
dm = DataManager()
mm = ModelManager()

# Включаем автологгирование
mlflow.keras.autolog()
warnings.filterwarnings("ignore", module="mlflow")  # Игнорируем предупреждения MLflow


2025/05/13 13:01:05 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.8.0, but the installed version is 3.9.2. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.


In [4]:
RANDOM_STATE = 42
BATCH_SIZE = 64
LEARNING_RATE = 1e-3

# Фиксируем все источники случайности
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# Для полной детерминированности (может замедлить GPU)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
tf.config.experimental.enable_op_determinism()

## 1. Загрузка данных

In [5]:
data_path = 'data'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')
train_data.shape

(8693, 14)

In [6]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


## 2. Предобработка данных 

In [7]:
# bad_columns = ['PassengerId', 'Name', ]
bad_columns = ['Name', ]

In [8]:
# Разделение на X / y
X, y = dm.split_data_set_to_x_y(train_data, 'Transported')
print(X.shape, y.shape)
X_submission = test_data.copy()
print(X_submission.shape)

(8693, 13) (8693,)
(4277, 13)


In [9]:
X.drop(columns=bad_columns, inplace=True)
X_submission.drop(columns=bad_columns, inplace=True)
print(X.shape, X_submission.shape)

(8693, 12) (4277, 12)


In [10]:
def make_feature_eng_great_again(train_X_in, test_X_in):
    """Хелпер, который создает фичи, логарифмирует и выравнивает колонки."""
    # Работаем с копиями, чтобы не изменять оригинальные X, X_test вне функции
    train_X = train_X_in.copy()
    test_X = test_X_in.copy()

    def create_features(df):
        # Колонка: Cabin
        # Уникальные значения: ['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
        # Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side,
        # where side can be either P for Port or S for Starboard.
        # Разделим признак на 3: Deck, Num, Side
        df['Deck'] = df['Cabin'].str.split('/').str[0]
        df['Num'] = df['Cabin'].str.split('/').str[1].astype(float)
        df['Side'] = df['Cabin'].str.split('/').str[2]
        df.drop(columns=['Cabin'], inplace=True)
        
        # PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is 
        # travelling with and pp is their number within the group. People in a group are often family members, but not always.
        df['Group'] = df['PassengerId'].str.split('_').str[0].astype(int)
        # df["GroupSize"] = df.groupby("Group")["Group"].transform("count")
        df.drop(columns=['PassengerId'], inplace=True)
        
        # # Indicates whether the passenger was traveling alone or not
        # df["Alone"] = df["GroupSize"] == 1
        
        #
        # # RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed
        # # at each of the Spaceship Titanic's many luxury amenities.
        # # Преобразуем в 1 колонку - TotalAmount
        df['TotalAmount'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
            
        return df

    # Создаем фичи
    train_X = create_features(train_X)
    test_X = create_features(test_X)
    print("Features created.")

    # Согласуем и сортируем колонки ПОСЛЕ всех манипуляций
    final_feature_cols = sorted(train_X.columns.tolist()) # Сортируем для стабильности
    train_X = train_X[final_feature_cols]
    test_X = test_X.reindex(columns=final_feature_cols, fill_value=0)
    print("Columns aligned and sorted.")

    return train_X, test_X

In [11]:
# Вызываем функцию с правильными данными (X, X_test)
X, X_submission = make_feature_eng_great_again(X, X_submission)

print("\nProcessing complete. Final shapes:")
print(f"X_processed: {X.shape}")
print(f"X_test_processed: {X_submission.shape}")

Features created.
Columns aligned and sorted.

Processing complete. Final shapes:
X_processed: (8693, 15)
X_test_processed: (4277, 15)


In [12]:
X.head()

,Age,CryoSleep,Deck,Destination,FoodCourt,Group,HomePlanet,Num,RoomService,ShoppingMall,Side,Spa,TotalAmount,VIP,VRDeck
0,39.0,False,B,TRAPPIST-1e,0.0,1,Europa,0.0,0.0,0.0,P,0.0,0.0,False,0.0
1,24.0,False,F,TRAPPIST-1e,9.0,2,Earth,0.0,109.0,25.0,S,549.0,736.0,False,44.0
2,58.0,False,A,TRAPPIST-1e,3576.0,3,Europa,0.0,43.0,0.0,S,6715.0,10383.0,True,49.0
3,33.0,False,A,TRAPPIST-1e,1283.0,3,Europa,0.0,0.0,371.0,S,3329.0,5176.0,False,193.0
4,16.0,False,F,TRAPPIST-1e,70.0,4,Earth,1.0,303.0,151.0,S,565.0,1091.0,False,2.0


In [13]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns
numeric_columns, non_numeric_columns,

(Index(['Age', 'FoodCourt', 'Group', 'Num', 'RoomService', 'ShoppingMall',
        'Spa', 'TotalAmount', 'VRDeck'],
       dtype='object'),
 Index(['CryoSleep', 'Deck', 'Destination', 'HomePlanet', 'Side', 'VIP'], dtype='object'))

## Делим данные на train/dev/test

In [14]:
# 1) Сначала отделяем тест (10%):
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y,
    test_size=0.10,
    random_state=42,
    stratify=y
)
# Параметр stratify в train_test_split отвечает за стратифицированное разбиение — то есть 
# разделение данных так, чтобы доля каждого класса (или другой целевой группы) в выборках
# train/val/test была примерно такая же, как в исходном датасете.

# 2) Из оставшихся берём валидацию (10% от всех → примерно 11% от X_train_full):
val_frac = 0.10 / 0.90  # ≈0.111…
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full,
    test_size=val_frac,
    random_state=42,
    stratify=y_train_full
)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

Train: 6953, Val: 870, Test: 870


## Строим пайплайн препроцессинга

In [15]:
# Пайплайн для числовых признаков (итеративное заполнение)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),  # разницы не дает
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- Объединяем препроцессоры ---
preprocessor = ColumnTransformer(
    transformers=[
        # Применяем к исходным числовым колонкам
        ('num', numeric_transformer, numeric_columns),
        # Применяем к исходным категориальным колонкам
        ('cat', categorical_transformer, non_numeric_columns),
    ],
    remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
    verbose_feature_names_out=False  # Чтобы имена колонок не менялись на 'num__colname' и т.д.
)

In [16]:
# --- Применяем препроцессор к данным ---
preprocessor.fit(X_train)
X_train = preprocessor.transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

Train: (6953, 29), Val: (870, 29), Test: (870, 29)


## Строим модель

In [17]:
# Переводим в tf.data.Dataset
def make_dataset(X, y=None, batch_size=64, shuffle=False):
    """Cтроит конвейер данных (tf.data.Dataset).
    
    Конвейер легко масштабируется и оптимизируется под тренинг в TensorFlow.
    from_tensor_slices() - Берёт два массива NumPy (X и y) и превращает их в объект Dataset, где каждый элемент — это кортеж (x_i, y_i).
	•	Под капотом:
	•	Не копирует сразу всё в память TensorFlow-сессии; создаёт «план» (граф) операций.
	•	При итерации берёт один срез из X и один из y и отдаёт дальше.

    Pro tip: при очень больших данных вместо массивов лучше потоком читать из файлов (например, tf.data.TextLineDataset или TFRecordDataset), чтобы не тащить всё в RAM. 
    
    prefetch - Автономно вытягивает и подготавливает следующий батч во время обучения на текущем.
	•	AUTOTUNE
    Дозирует объём буфера сам, чтобы сбалансировать загрузку CPU (или I/O) и GPU.
    •	Под капотом:
    Запускает отдельный фон-поток (или пул), который заранее выполняет все операции конвейера вплоть до батчинга, чтобы когда модель попросит данные, они уже лежали готовы в памяти.
    
    .batch(batch_size) - группирует образцы в мини-батчи размером batch_size. Вместо того, чтобы отдавать по одному образцу, датасет будет отдавать тензор формы (batch_size, ...) и каждая итерация обучения в model.fit будет происходить на этом куске данных. 
    """
    if y is None:
        # Если y нет, то создаем датасет только из X
        ds = tf.data.Dataset.from_tensor_slices(X.astype('float32'))
    else:
        ds = tf.data.Dataset.from_tensor_slices((X.astype('float32'), y.astype('float32')))
        if shuffle:
            ds = ds.shuffle(buffer_size=len(X), seed=RANDOM_STATE)
    
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [18]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(6953, 29) (870, 29) (870, 29)
(6953,) (870,) (870,)


In [19]:
# Пишем repeat, чтобы не было out_of_range ошибки из-за батчей
# train_ds = make_dataset(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True).repeat()
train_ds = make_dataset(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
val_ds = make_dataset(X_val, y_val, batch_size=BATCH_SIZE)
test_ds = make_dataset(X_test, y_test, batch_size=BATCH_SIZE)

In [20]:
def create_model(input_shape, random_state=42):
    tf.random.set_seed(random_state)
    
    initializer = GlorotUniform(seed=random_state)
    
    inputs = tf.keras.Input(shape=(input_shape,), name='features')
    
    x = inputs
    layer_sizes = [256, 128, 64, 32, 16, 8]
    dropout_rates = [0.3, 0.3, 0.2, 0, 0, 0]
    
    for i, (size, dropout_rate) in enumerate(zip(layer_sizes, dropout_rates)):
        x = Dense(size, kernel_initializer=initializer, kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization()(x)
        x = Activation('gelu')(x)
        if dropout_rate > 0:
            x = Dropout(dropout_rate, seed=random_state + i)(x)  # Разные seed для разных слоев
    
    outputs = Dense(1, activation='sigmoid', kernel_initializer=initializer)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Компилируем с фиксированным optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    # optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-5)
    optimizer.build(model.trainable_variables)
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

In [21]:
def train_model(
        shape,
        ds,
        ds_val,
        epochs=100,
        early_stopping_patience=10,
        reduce_lr_factor=0.5,
        reduce_lr_patience=5,
        verbose=2,
        experiment_name="classification experiment",
        run_name="DNN - Baseline DNN",
):
    # если есть открытый run — закроем его
    active = mlflow.active_run()
    if active is not None and active.info.lifecycle_stage == 'active':
        mlflow.end_run()
    
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=run_name) as run:  # Сохраняем run для логирования артефактов
    
        # Обучение с колбэками
        # ReduceLROnPlateau callback отслеживает указанную метрику (по умолчанию — val_loss) и, 
        # если она не улучшается в течение заданного числа эпох, автоматически уменьшает 
        # скорость обучения.
        callbacks = [  # Callbacks для контроля переобучения
            tf.keras.callbacks.EarlyStopping(patience=early_stopping_patience, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(
                factor=reduce_lr_factor,  # во сколько раз уменьшить lr (0.5 → lr_new = lr_old * 0.5)
                patience=reduce_lr_patience,  # ждать 5 эпох без улучшения
                min_lr=1e-6, # (опционально) не опускать lr ниже этого порога
                verbose=1  # выводить в логи сообщение об уменьшении
            )
        ]
        
        new_model = create_model(shape, random_state=RANDOM_STATE)
        model_history = new_model.fit(
            ds,
            epochs=epochs,
            validation_data = ds_val,
            callbacks=callbacks,
            verbose=verbose
        )
        
        return new_model, model_history

In [22]:
model, history = train_model(
    shape=X_train.shape[1],
    ds=train_ds,
    ds_val=val_ds,
    epochs=100,
    early_stopping_patience=10,
    reduce_lr_factor=0.5,
    reduce_lr_patience=5,
    verbose=2,
    experiment_name="classification experiment",
    run_name="DNN - Default Baseline"
) 

Epoch 1/300
109/109 - 2s - 17ms/step - accuracy: 0.7208 - auc: 0.7999 - loss: 0.9158 - val_accuracy: 0.7966 - val_auc: 0.8783 - val_loss: 0.8790 - learning_rate: 0.0010
Epoch 2/300
109/109 - 0s - 2ms/step - accuracy: 0.7812 - auc: 0.8647 - loss: 0.8085 - val_accuracy: 0.8138 - val_auc: 0.8893 - val_loss: 0.7753 - learning_rate: 0.0010
Epoch 3/300
109/109 - 0s - 2ms/step - accuracy: 0.7956 - auc: 0.8776 - loss: 0.7687 - val_accuracy: 0.8092 - val_auc: 0.8944 - val_loss: 0.7380 - learning_rate: 0.0010
Epoch 4/300
109/109 - 0s - 2ms/step - accuracy: 0.7949 - auc: 0.8810 - loss: 0.7444 - val_accuracy: 0.8195 - val_auc: 0.8969 - val_loss: 0.7113 - learning_rate: 0.0010
Epoch 5/300
109/109 - 0s - 2ms/step - accuracy: 0.7971 - auc: 0.8840 - loss: 0.7215 - val_accuracy: 0.8218 - val_auc: 0.8989 - val_loss: 0.6917 - learning_rate: 0.0010
Epoch 6/300
109/109 - 0s - 2ms/step - accuracy: 0.7984 - auc: 0.8841 - loss: 0.7039 - val_accuracy: 0.8195 - val_auc: 0.9021 - val_loss: 0.6697 - learning_rate

In [23]:
# После выбора лучших весов оцениваем на тесте (для честной оценки финальной модели.)
test_loss, test_acc, test_auc = model.evaluate(test_ds, verbose=2)
print(f"Test  — loss: {test_loss:.4f}, acc: {test_acc:.4f}, auc: {test_auc:.4f}")

14/14 - 0s - 1ms/step - accuracy: 0.8184 - auc: 0.9174 - loss: 0.3991
Test  — loss: 0.3991, acc: 0.8184, auc: 0.9174


In [25]:
# Стремимся к

# 14/14 - 0s - 2ms/step - accuracy: 0.8276 - auc: 0.9212 - loss: 0.381714/14 - 0s - 2ms/step - accuracy: 0.8276 - auc: 0.9212 - loss: 0.3817
# Test  — loss: 0.3817, acc: 0.8276, auc: 0.9212

## Делаем сабмишн

### Сабмишн делаем из той модели, которая обучалась на валидации

In [25]:
X_submission.shape

(4277, 15)

In [25]:
X_submission_processed = preprocessor.transform(X_submission)
submission_ds = make_dataset(X_submission_processed, None, batch_size=64)

In [26]:
# Получаем вероятности
probs = model.predict(submission_ds)      # shape = (num_samples, 1)
probs = probs.ravel()               # shape = (num_samples,)
probs

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([0.5425285 , 0.03048058, 0.997546  , ..., 0.9732298 , 0.7276377 ,
       0.56698316], dtype=float32)

In [27]:
# Конвертация в булевы метки (threshold=0.5)
labels = probs > 0.5                # array of True/False
labels, len(labels)

(array([ True, False,  True, ...,  True,  True,  True]), 4277)

In [28]:
# Собираем submission
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Transported': labels
})
submission.to_csv('DNN_submission_v6.csv', index=False)
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


### Сабмишн делаем, обучая модель на всех данных разом

In [26]:
def train_model_final(
        shape,
        ds,
        epochs=100,
        reduce_lr_factor=0.5,
        reduce_lr_patience=10,  # Увеличиваем patience, так как нет валидации
        verbose=2,
        experiment_name="classification experiment",
        run_name="DNN - Final Model Submission",
):
    """Обучение модели на всех данных с ограниченными колбэками"""
    
    # Если есть открытый run — закроем его
    active = mlflow.active_run()
    if active is not None and active.info.lifecycle_stage == 'active':
        mlflow.end_run()
    
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=run_name) as run:
        
        # Только колбэки, которые не требуют валидационных данных
        callbacks = [
            # ReduceLROnPlateau может работать с loss (а не val_loss)
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='loss',  # Изменили на loss вместо val_loss
                factor=reduce_lr_factor,
                patience=reduce_lr_patience,
                min_lr=1e-6,
                verbose=1
            )
        ]
        
        new_model = create_model(shape, random_state=RANDOM_STATE)
        model_history = new_model.fit(
            ds,
            epochs=epochs,
            callbacks=callbacks,
            verbose=verbose
        )
        
        return new_model, model_history

In [27]:
# Сначала обучим модель на ВСЕХ данных
preprocessor.fit(X)
X_processed = preprocessor.transform(X)
X_submission_processed = preprocessor.transform(X_submission)

print(f"Формы данных после препроцессинга:")
print(X_processed.shape, X_submission_processed.shape)

Формы данных после препроцессинга:
(8693, 29) (4277, 29)


In [28]:
final_train_ds = make_dataset(X_processed, y, batch_size=BATCH_SIZE, shuffle=True)
submission_ds = make_dataset(X_submission_processed, None, batch_size=BATCH_SIZE)

In [32]:
# Из истории валидации находим оптимальное количество эпох
best_epoch = np.argmin(history.history['val_loss']) + 1  # +1 так как индексация с 0
# Добавляем относительный запас (например, 20% больше)
final_epochs = int(best_epoch * 1.2)
print(f"Best validation epoch: {best_epoch}")
print(f"Final training epochs: {final_epochs}")

Best validation epoch: 22
Final training epochs: 26


In [30]:
# Обучаем модель на всех данных
final_model, _ = train_model_final(
    shape=X_processed.shape[1],
    ds=final_train_ds,
    epochs=best_epoch,
    experiment_name="classification experiment",
    run_name=f"DNN - Final Submission ({final_epochs} epochs)"
) 

Epoch 1/32
136/136 - 2s - 11ms/step - accuracy: 0.7271 - auc: 0.8047 - loss: 0.5834 - learning_rate: 0.0010
Epoch 2/32
136/136 - 0s - 2ms/step - accuracy: 0.7860 - auc: 0.8700 - loss: 0.4877 - learning_rate: 0.0010
Epoch 3/32
136/136 - 0s - 2ms/step - accuracy: 0.7924 - auc: 0.8791 - loss: 0.4706 - learning_rate: 0.0010
Epoch 4/32
136/136 - 0s - 2ms/step - accuracy: 0.7913 - auc: 0.8820 - loss: 0.4632 - learning_rate: 0.0010
Epoch 5/32
136/136 - 0s - 2ms/step - accuracy: 0.7998 - auc: 0.8877 - loss: 0.4527 - learning_rate: 0.0010
Epoch 6/32
136/136 - 0s - 2ms/step - accuracy: 0.8048 - auc: 0.8902 - loss: 0.4479 - learning_rate: 0.0010
Epoch 7/32
136/136 - 0s - 2ms/step - accuracy: 0.8059 - auc: 0.8922 - loss: 0.4436 - learning_rate: 0.0010
Epoch 8/32
136/136 - 0s - 2ms/step - accuracy: 0.8094 - auc: 0.8953 - loss: 0.4381 - learning_rate: 0.0010
Epoch 9/32
136/136 - 0s - 2ms/step - accuracy: 0.8079 - auc: 0.8954 - loss: 0.4352 - learning_rate: 0.0010
Epoch 10/32
136/136 - 0s - 2ms/step 

In [31]:
# Получаем вероятности
probs = final_model.predict(submission_ds)      # shape = (num_samples, 1)
probs = probs.ravel()               # shape = (num_samples,)

# Конвертация в булевы метки (threshold=0.5)
labels = probs > 0.5                # array of True/False
labels, len(labels)

# Собираем submission
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Transported': labels
})
submission.to_csv('DNN_submission_v22.csv', index=False)
submission.head()

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [24]:
# Надо улучшить optuna, чтобы она делала подбор, основываясь на разных валидационных срезах, потому что ВРУЧНУЮ подбирать - нереально